<a href="https://colab.research.google.com/github/kr7/seizure/blob/main/SeizureDetection_ROCKET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Activity Recognition Based on Accelerometer Data with Enhanced ROCKET Algorithm**

In [ ]:
# dataset may be set to "Epilepsy" or "OpenSeizure"
# - "Epilepsy" denotes the dataset that is publicly available from
#   https://timeseriesclassification.com/aeon-toolkit/Epilepsy.zip
# - If you want to use the "OpenSeizure" data, you should first execute
#   the script to preprocess data. This notebook will ask you to upload
#   the preprocessed data.
dataset = "Epilepsy"

# Set it to True in order to generate univariate convolutional kernels and
# to assign them to channels randomly
UNIVARIATE = False

In [ ]:
import numpy as np
import os
from numpy import random as rnd
from scipy.io import arff
from scipy.stats import ttest_rel
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import RidgeClassifier

In [ ]:
if dataset == "Epilepsy":
  os.system("wget https://timeseriesclassification.com/aeon-toolkit/Epilepsy.zip")
  os.system("unzip -q Epilepsy.zip")

  # In order to perform 10-fold cross-validation, we
  # will merge the provided train and test splits and
  # we will split the data durign the cross-validation

  NUM_CLASSES = 4

  X = []
  y = []

  for filename in ['Epilepsy_TRAIN.arff', 'Epilepsy_TEST.arff' ]:
    raw_data = arff.loadarff(filename)

    nextInstance = True
    i = 0

    while nextInstance:
      try:
        dim1 = list(raw_data[0][i][0][0])
        dim2 = list(raw_data[0][i][0][1])
        dim3 = list(raw_data[0][i][0][2])
        label = raw_data[0][i][1]

        X.append([dim1,dim2,dim3])
        if label == b'EPILEPSY':
          label = 0
        elif label == b'WALKING':
          label = 1
        elif label == b'RUNNING':
          label = 2
        elif label == b'SAWING':
          label = 3
        else:
          print(f'Unexpected label: {label}')
        y.append(label)

        i = i+1
      except:
        nextInstance = False

  X = np.array(X)
  y = np.array(y)
elif dataset == "OpenSeizure":
  from google.colab import files
  uploaded = files.upload()

  # Read (preprocessed) OpenSeizure data

  NUM_CLASSES = 2

  X = []
  y = []

  with open('OpenSeizure.txt', 'r') as f:
    nextInstance = True
    while nextInstance:
      try:
        dim1 = [ float(v) for v in f.readline().split(",")]
        dim2 = [ float(v) for v in f.readline().split(",")]
        dim3 = [ float(v) for v in f.readline().split(",")]
        label = int(f.readline())

        X.append([dim1,dim2,dim3])
        y.append(label)
      except:
        nextInstance = False

  X = np.array(X)
  y = np.array(y)
else:
  raise Exception(f"Unknown dataset: {dataset}")

In [ ]:
NUM_CHANNELS = 3

In [ ]:
%load_ext cython

In [ ]:
%%cython

cimport cython
from libc.stdlib cimport malloc, free


def dtw(ts1p, ts2p):
    cdef int LEN_TS1
    cdef int LEN_TS2
    cdef int i
    cdef int j
    cdef float * ts1
    cdef float * ts2
    cdef float * dtw_matrix
    cdef float d

    LEN_TS1 = len(ts1p)
    LEN_TS2 = len(ts2p)

    ts1 = <float *> malloc(LEN_TS1*cython.sizeof(float))
    ts2 = <float *> malloc(LEN_TS2*cython.sizeof(float))
    dtw_matrix = <float *> malloc(LEN_TS1*LEN_TS2*cython.sizeof(float))
      # this is a flattened DTW matrix dtw_matrix[i,j] -> dtw_matrix[i*LEN_TS2 + j]
    if ts1 is NULL or ts2 is NULL:
      raise MemoryError()
    for i in xrange(LEN_TS1):
      ts1[i] = ts1p[i]
    for i in xrange(LEN_TS2):
      ts2[i] = ts2p[i]

    dtw_matrix[0] = abs(ts1[0]-ts2[0])

    for i in range(1, LEN_TS1):
      dtw_matrix[i*LEN_TS2] = dtw_matrix[(i-1)*LEN_TS2]+abs(ts1[i]-ts2[0])

    for j in range(1, LEN_TS2):
      dtw_matrix[j] = dtw_matrix[j-1]+abs(ts1[0]-ts2[j])

    for i in range(1, LEN_TS1):
      for j in range(1, LEN_TS2):
        dtw_matrix[i*LEN_TS2+j] = min(dtw_matrix[(i-1)*LEN_TS2+j-1],
                                      dtw_matrix[(i-1)*LEN_TS2+j],
                                      dtw_matrix[i*LEN_TS2+j-1]) + abs(ts1[i]-ts2[j])

    d=dtw_matrix[ LEN_TS1*LEN_TS2-1 ]

    free(ts1)
    free(ts2)
    free(dtw_matrix)

    return d

In [ ]:
def log2(x):
  return np.log(x)/np.log(2)


def get_random_convolution_params(seed, l_input):
  rnd.seed(seed)
  length = rnd.randint(0,2)*2+7
  weights = rnd.normal(0,1, (NUM_CHANNELS, length) )
  bias = float(rnd.uniform(-1, 1))
  max_exp = np.floor(log2((l_input-1)/(length-1)))-1
  dilation = int( 2**(rnd.uniform(0,max_exp)) )
  padding = rnd.randint(0,1)
  return (length, weights, bias, dilation, padding)


def get_random_convolution_params_univariate(seed, l_input):
  rnd.seed(seed)
  length = rnd.randint(0,2)*2+7
  weights = rnd.normal(0,1, (length) )
  bias = float(rnd.uniform(-1, 1))
  max_exp = np.floor(log2((l_input-1)/(length-1)))-1
  dilation = int( 2**(rnd.uniform(0,max_exp)) )
  padding = rnd.randint(0,1)
  channel = rnd.randint(0,NUM_CHANNELS-1)
  return (length, weights, bias, dilation, padding, channel)

def apply_convolution( time_series, convolution ):
  length, weights, bias, dilation, padding = convolution
  window_size = length*dilation
  time_series = list(time_series)

  if padding == 1:
    zeros = [0]*(dilation*(length-1)/2)
    for i in range(NUM_CHANNELS):
      time_series[i] = zeros + time_series[i] + zeros

  segments = np.array(
      [ [ time_series[j][i:i+window_size:dilation] for j in range(NUM_CHANNELS) ]
        for i in range(0, int(len(time_series[0])-window_size)) ])

  conv = np.array([np.sum(s*weights) + bias for s in segments])
  dtw_conv = np.array( [ np.sum([dtw(s[c], weights[c]) for c in range(len(s))]) for s in segments] )

  return conv, dtw_conv


def apply_convolution_univariate( time_series, convolution ):
  length, weights, bias, dilation, padding, channel = convolution
  window_size = length*dilation
  time_series = list(time_series)

  if padding == 1:
    zeros = [0]*(dilation*(length-1)/2)
    for i in range(NUM_CHANNELS):
      time_series[i] = zeros + time_series[i] + zeros

  segments = np.array(
      [ time_series[channel][i:i+window_size:dilation]
        for i in range(0, int(len(time_series[0])-window_size)) ])

  conv = np.array([np.sum(s*weights) + bias for s in segments])
  dtw_conv = np.array( [ dtw(s, weights) for s in segments] )

  return conv, dtw_conv


def ppv(series):
  return float(np.sum(series > 0) / np.size(series))


def get_rocket_features(time_series_dataset, convolutional_filters,
                        f_convolution = apply_convolution):
  dataset_features_max = [] # features with conventional convolution
  dataset_features_ppv = [] # features with conventional convolution
  dataset_features_dtw_min = [] # features with dynamic convolution
  i = 0
  for ts in time_series_dataset:
    if i%1==0:
      print(f"{i:4}/{len(time_series_dataset)}")
    i = i+1

    ts_features_max = []
    ts_features_ppv = []
    ts_features_dtw_min = []
    for c in convolutional_filters:
      convolved_ts, dtw_convolved_ts = f_convolution(ts, c)
      ts_features_max.append( float(max(convolved_ts)) )
      ts_features_ppv.append( ppv(convolved_ts) )
      ts_features_dtw_min.append( float(min(dtw_convolved_ts)) )
       # PPV does not make sense for DTW!
    dataset_features_max.append( ts_features_max )
    dataset_features_ppv.append( ts_features_ppv )
    dataset_features_dtw_min.append( ts_features_dtw_min )
  return np.array(dataset_features_max), np.array(dataset_features_ppv), \
    np.array(dataset_features_dtw_min)

In [ ]:
def evaluate(pred, y_test):
  err = np.mean(pred != y_test)
  tp, tn, fp, fn = 0, 0, 0, 0
  for i in range(len(pred)):
    if pred[i] == 0 and y_test[i] == 0:
      tp = tp+1
    elif pred[i] == 0 and y_test[i] != 0:
      fp = fp+1
    elif pred[i] != 0 and y_test[i] == 0:
      fn = fn+1
    elif pred[i] != 0 and y_test[i] != 0:
      tn = tn+1
    else:
      raise Exception("Bug!")

  if tp == 0:
    prec = 0
    recall = 0
  else:
    prec = tp / (tp+fp)
    recall = tp / (tp+fn)
  if (prec == 0) and (recall == 0):
    f1 = 0
  else:
    f1 = 2*prec*recall/(prec+recall)

  return err, prec, recall, f1, tp, fp, tn, fn

In [ ]:
# Convolutional filters are independent of the data (as their parameters are
# sampled from random distributions), therefore the features based on
# convolutional filters may be pre-calculated (so that they do not need to be
# calculated in each round of the cross-validation which speeds up the
# execution of the experiment)

length = len(X[0][0])
convolutional_filters = []
if UNIVARIATE:
  for i in range(10000):
    convolutional_filters.append(get_random_convolution_params_univariate(i, length))

  features_max, features_ppv, features_dtw_min = \
    get_rocket_features(X, convolutional_filters, apply_convolution_univariate)
else:
  for i in range(10000):
    convolutional_filters.append(get_random_convolution_params(i, length))

  features_max, features_ppv, features_dtw_min = \
    get_rocket_features(X, convolutional_filters)



In [ ]:
# 10-times 10-fold cross-validation

all_err_rocket  = np.zeros( (8, 100) )
all_err_ppv     = np.zeros( (8, 100) )
all_err_max     = np.zeros( (8, 100) )
all_err_erocket = np.zeros( (8, 100) )

fold = 0

for seed in range(10):
  kf = StratifiedKFold(n_splits=10, random_state=seed+42, shuffle=True)
  for train_index, test_index in kf.split(X, y):
    print(f"FOLD: {fold:2}")
    features_train_max = features_max[train_index]
    features_train_ppv = features_ppv[train_index]
    features_train_dtw_min = features_dtw_min[train_index]

    features_test_max = features_max[test_index]
    features_test_ppv = features_ppv[test_index]
    features_test_dtw_min = features_dtw_min[test_index]

    y_train = y[train_index]
    y_test  = y[test_index]

    cls = RidgeClassifier()
    cls.fit( np.hstack((features_train_max, features_train_ppv)), y_train)
    pred = cls.predict( np.hstack((features_test_max, features_test_ppv)) )
    all_err_rocket[:,fold] = evaluate(pred, y_test)

    cls = RidgeClassifier()
    cls.fit( features_train_ppv, y_train )
    pred = cls.predict( features_test_ppv )
    all_err_ppv[:,fold] = evaluate(pred, y_test)

    cls = RidgeClassifier()
    cls.fit( features_train_max, y_train )
    pred = cls.predict( features_test_max )
    all_err_max[:,fold] = evaluate(pred, y_test)

    cls = RidgeClassifier()
    cls.fit( features_train_dtw_min, y_train )
    pred = cls.predict( features_test_dtw_min )
    all_err_erocket[:,fold] = evaluate(pred, y_test)

    fold = fold + 1

In [ ]:
print( "Method                    Classification error  p-values")
print(f"ROCKET                    "+
      f"{np.mean(all_err_rocket[0]):6.4f} +/- {np.std(all_err_rocket[0]):6.4f}  ")
print(f"ROCKET-PPV                "+
      f"{np.mean(all_err_ppv[0]):6.4f} +/- {np.std(all_err_ppv[0]):6.4f}  ")
print(f"ROCKET-MAX                "+
      f"{np.mean(all_err_max[0]):6.4f} +/- {np.std(all_err_max[0]):6.4f}  ")
print(f"EROCKET (our approach)    "+
      f"{np.mean(all_err_erocket[0]):6.4f} +/- {np.std(all_err_erocket[0]):6.4f}     "+
      f"{ttest_rel(all_err_rocket[0], all_err_erocket[0])[1]:5.3f}  "+
      f"{ttest_rel(all_err_ppv[0], all_err_erocket[0])[1]:5.3f}  "+
      f"{ttest_rel(all_err_max[0], all_err_erocket[0])[1]:5.3f}  ")

In [ ]:
print("all_err_rocket=np."+repr(all_err_rocket))
print("all_err_ppv=np."+repr(all_err_ppv))
print("all_err_max=np."+repr(all_err_max))
print("all_err_erocket=np."+repr(all_err_erocket))